In [125]:
# Librerias
from google.colab import drive
import pickle

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, Flatten, MaxPooling2D, BatchNormalization, LSTM

import matplotlib.pyplot as plt

In [28]:
drive.mount('/content/gdrive') # WARNING! Pedirá permisos
data = open('/content/gdrive/MyDrive/Colab Notebooks/mfcc.pickle','rb')
data = pickle.load(data)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
data.keys()

dict_keys(['spp', 'mfcc'])

In [162]:
x = np.array(data['mfcc'])

In [164]:
y_encoder = LabelEncoder()
y_encoder.fit(data['spp'])
y = y_encoder.transform(data['spp'])

In [165]:
y

array([15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [166]:
# create train, validation and test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.30)

# add an axis to input sets
# x_train = x_train[..., np.newaxis]
# x_validation = x_validation[..., np.newaxis]
# x_test = x_test[..., np.newaxis]

In [167]:
shape_1 = x_train.shape[1]
shape_2 = x_train.shape[2]

x_train = x_train.reshape(x_train.shape[0], shape_1, shape_2, 1)
x_test = x_test.reshape(x_test.shape[0], shape_1, shape_2, 1)

input_shape = (shape_1,shape_2,1)
input_shape

(95, 128, 1)

In [172]:
# build network topology
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(12, (3,3), activation='relu'))
model.add(MaxPooling2D((3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(32, (2,2), activation='relu'))
model.add(MaxPooling2D((3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
        
model.add(Flatten())
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='softmax'))

In [173]:
# compile model
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 93, 126, 16)       160       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 47, 63, 16)       0         
 g2D)                                                            
                                                                 
 batch_normalization_39 (Bat  (None, 47, 63, 16)       64        
 chNormalization)                                                
                                                                 
 conv2d_41 (Conv2D)          (None, 45, 61, 12)        1740      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 23, 31, 12)       0         
 g2D)                                                            
                                                     

In [174]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
6/6 [==============================] - 2s 239ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 2/10
6/6 [==============================] - 1s 235ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 3/10
6/6 [==============================] - 1s 233ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 4/10
6/6 [==============================] - 1s 240ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 5/10
6/6 [==============================] - 1s 235ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 6/10
6/6 [==============================] - 1s 240ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 7/10
6/6 [==============================] - 1s 237ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 8/10
6/6 [==============================] - 1s 234ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 9/10
6/6 [==============================] - 1s 235ms/step - loss: 0.0000e+00 - accuracy: 0.0994
Epoch 10/10
6/6 [==============================] - 1s 235ms/step - loss: 0.0000e+0